In [1]:
include("cosmology_vars.jl")
include("nu_osc_params.jl")

using DelimitedFiles
using Plots
using Interpolations
using LaTeXStrings
using QuadGK
using SpecialFunctions
using BenchmarkTools;

In [117]:
function F0_tint_func(F0vec::Vector{Float64})
    es = range(0, 100, 2000)
    F0int_ne = Interpolations.interpolate((vec(es),), F0vec, Gridded(Linear()))
    return extrapolate(F0int_ne, 0.0)
end

F0s_vec = readdlm("F0s_vec.txt", comments=true)

F0_νe_270sm, F0_νebar_270sm, F0_νx_270sm = F0_tint_func(vec(F0s_vec[:, 1])), F0_tint_func(vec(F0s_vec[:, 2])), F0_tint_func(vec(F0s_vec[:, 3]))
F0_νe_112sm, F0_νebar_112sm, F0_νx_112sm = F0_tint_func(vec(F0s_vec[:, 4])), F0_tint_func(vec(F0s_vec[:, 5])), F0_tint_func(vec(F0s_vec[:, 6]))
F0_νe_bh, F0_νebar_bh, F0_νx_bh = F0_tint_func(vec(F0s_vec[:, 7])), F0_tint_func(vec(F0s_vec[:, 8])), F0_tint_func(vec(F0s_vec[:, 9]))

function F0(E, β, sm)
    if β == "e" && sm == "small"
        return F0_νe_112sm(E)
    elseif β == "e" && sm == "large"
        return F0_νe_270sm(E)
    elseif β == "e" && sm == "bh"
        return F0_νe_bh(E)
    elseif β == "ebar" && sm == "small"
        return F0_νebar_112sm(E)
    elseif β == "ebar" && sm == "large"
        return F0_νebar_270sm(E)
    elseif β == "ebar" && sm == "bh"
        return F0_νebar_bh(E)   
    elseif β == "x" && sm == "small"
        return F0_νx_112sm(E)
    elseif β == "x" && sm == "large"
        return F0_νx_270sm(E)
    elseif β == "x" && sm == "bh"
        return F0_νx_bh(E)
    else
        return 0
    end
end

# Oscillations thru the SN medium
# Accounting for oscillations thru the SN medium

s12 = 0.297
c12 = 1 - s12
PH = 0

# ordering = "NO" (normal ordering) or "IO" (inverted ordering)
function F(E, β, sm, ordering)
    if ordering == "NO"
        if β == "e"
            return F0(E, "x", sm)
        elseif β == "ebar"
            return c12*F0(E, "ebar", sm) + s12*F0(E, "x", sm)
        elseif β == "x"
            return 0.5*(F0(E, "e", sm) + F0(E, "x", sm))
        elseif β == "xbar"
            return 0.5*(s12*F0(E, "ebar", sm) + (1 + c12)*F0(E, "x", sm))
        else
            return 0
        end
    elseif ordering == "IO"
        if β == "e"
            return s12*F0(E, "e", sm) + c12*F0(E, "x", sm)
        elseif β == "ebar"
            return F0(E, "x", sm)
        elseif β == "x"
            return 0.5*(c12*F0(E, "e", sm) + (1 + s12)*F0(E, "x", sm))
        elseif β == "xbar"
            return 0.5*(F0(E, "ebar", sm) + F0(E, "x", sm))
        else
            return 0
        end
    else
        return 0
    end
end

# In the mass basis now: i = 1, 2, 3, nubar = true or false
function Fmass_old(E, i, sm, ordering, nubar)
    if nubar==false
        return Usqred(ordering)[1, i]*F(E, "e", sm, ordering) + (1 - Usqred(ordering)[1, i])*F(E, "x", sm, ordering)
    elseif nubar==true
        return Usqred(ordering)[1, i]*F(E, "ebar", sm, ordering) + (1 - Usqred(ordering)[1, i])*F(E, "xbar", sm, ordering)
    else
        return 0
    end
end;

In [119]:
function Fmass_tint_func(i, sm, ordering, nubar)
    es = range(0, 100, 2000)
    Fmassint_ne = Interpolations.interpolate((vec(es),), Fmass_old.(es, i, sm, ordering, nubar), Gridded(Linear()))
    return extrapolate(Fmassint_ne, 0.0)
end

Fm_1_112sm_NO_nu = Fmass_tint_func(1, "small", "NO", false)
Fm_1_112sm_NO_nubar = Fmass_tint_func(1, "small", "NO", true)
Fm_1_112sm_IO_nu = Fmass_tint_func(1, "small", "IO", false)
Fm_1_112sm_IO_nubar = Fmass_tint_func(1, "small", "IO", true)
Fm_1_270sm_NO_nu = Fmass_tint_func(1, "large", "NO", false)
Fm_1_270sm_NO_nubar = Fmass_tint_func(1, "large", "NO", true)
Fm_1_270sm_IO_nu = Fmass_tint_func(1, "large", "IO", false)
Fm_1_270sm_IO_nubar = Fmass_tint_func(1, "large", "IO", true)
Fm_1_bh_NO_nu = Fmass_tint_func(1, "bh", "NO", false)
Fm_1_bh_NO_nubar = Fmass_tint_func(1, "bh", "NO", true)
Fm_1_bh_IO_nu = Fmass_tint_func(1, "bh", "IO", false)
Fm_1_bh_IO_nubar = Fmass_tint_func(1, "bh", "IO", true)

Fm_2_112sm_NO_nu = Fmass_tint_func(2, "small", "NO", false)
Fm_2_112sm_NO_nubar = Fmass_tint_func(2, "small", "NO", true)
Fm_2_112sm_IO_nu = Fmass_tint_func(2, "small", "IO", false)
Fm_2_112sm_IO_nubar = Fmass_tint_func(2, "small", "IO", true)
Fm_2_270sm_NO_nu = Fmass_tint_func(2, "large", "NO", false)
Fm_2_270sm_NO_nubar = Fmass_tint_func(2, "large", "NO", true)
Fm_2_270sm_IO_nu = Fmass_tint_func(2, "large", "IO", false)
Fm_2_270sm_IO_nubar = Fmass_tint_func(2, "large", "IO", true)
Fm_2_bh_NO_nu = Fmass_tint_func(2, "bh", "NO", false)
Fm_2_bh_NO_nubar = Fmass_tint_func(2, "bh", "NO", true)
Fm_2_bh_IO_nu = Fmass_tint_func(2, "bh", "IO", false)
Fm_2_bh_IO_nubar = Fmass_tint_func(2, "bh", "IO", true)

Fm_3_112sm_NO_nu = Fmass_tint_func(3, "small", "NO", false)
Fm_3_112sm_NO_nubar = Fmass_tint_func(3, "small", "NO", true)
Fm_3_112sm_IO_nu = Fmass_tint_func(3, "small", "IO", false)
Fm_3_112sm_IO_nubar = Fmass_tint_func(3, "small", "IO", true)
Fm_3_270sm_NO_nu = Fmass_tint_func(3, "large", "NO", false)
Fm_3_270sm_NO_nubar = Fmass_tint_func(3, "large", "NO", true)
Fm_3_270sm_IO_nu = Fmass_tint_func(3, "large", "IO", false)
Fm_3_270sm_IO_nubar = Fmass_tint_func(3, "large", "IO", true)
Fm_3_bh_NO_nu = Fmass_tint_func(3, "bh", "NO", false)
Fm_3_bh_NO_nubar = Fmass_tint_func(3, "bh", "NO", true)
Fm_3_bh_IO_nu = Fmass_tint_func(3, "bh", "IO", false)
Fm_3_bh_IO_nubar = Fmass_tint_func(3, "bh", "IO", true)

function Fmass_test1(E, sm, ordering, nubar)
    if sm == "small"
        if ordering == "NO"
            if nubar == false
                return Fm_1_112sm_NO_nu(E)
            elseif nubar == true
                return Fm_1_112sm_NO_nubar(E)
            else
                return 0.0
            end
        elseif ordering == "IO"
            if nubar == false
                return Fm_1_112sm_IO_nu(E)
            elseif nubar == true
                return Fm_1_112sm_IO_nubar(E)
            else
                return 0.0
            end
        else
            return 0.0
        end
    elseif sm == "large"
        if ordering == "NO"
            if nubar == false
                return Fm_1_270sm_NO_nu(E)
            elseif nubar == true
                return Fm_1_270sm_NO_nubar(E)
            else
                return 0.0
            end
        elseif ordering == "IO"
            if nubar == false
                return Fm_1_270sm_IO_nu(E)
            elseif nubar == true
                return Fm_1_270sm_IO_nubar(E)
            else
                return 0.0
            end
        else
            return 0.0
        end
    elseif sm == "bh"
        if ordering == "NO"
            if nubar == false
                return Fm_1_bh_NO_nu(E)
            elseif nubar == true
                return Fm_1_bh_NO_nubar(E)
            else
                return 0.0
            end
        elseif ordering == "IO"
            if nubar == false
                return Fm_1_bh_IO_nu(E)
            elseif nubar == true
                return Fm_1_bh_IO_nubar(E)
            else
                return 0.0
            end
        else
            return 0.0
        end
    else
        return 0.0
    end
end
function Fmass_test2(E, sm, ordering, nubar)
    if sm == "small"
        if ordering == "NO"
            if nubar == false
                return Fm_2_112sm_NO_nu(E)
            elseif nubar == true
                return Fm_2_112sm_NO_nubar(E)
            else
                return 0.0
            end
        elseif ordering == "IO"
            if nubar == false
                return Fm_2_112sm_IO_nu(E)
            elseif nubar == true
                return Fm_2_112sm_IO_nubar(E)
            else
                return 0.0
            end
        else
            return 0.0
        end
    elseif sm == "large"
        if ordering == "NO"
            if nubar == false
                return Fm_2_270sm_NO_nu(E)
            elseif nubar == true
                return Fm_2_270sm_NO_nubar(E)
            else
                return 0.0
            end
        elseif ordering == "IO"
            if nubar == false
                return Fm_2_270sm_IO_nu(E)
            elseif nubar == true
                return Fm_2_270sm_IO_nubar(E)
            else
                return 0.0
            end
        else
            return 0.0
        end
    elseif sm == "bh"
        if ordering == "NO"
            if nubar == false
                return Fm_2_bh_NO_nu(E)
            elseif nubar == true
                return Fm_2_bh_NO_nubar(E)
            else
                return 0.0
            end
        elseif ordering == "IO"
            if nubar == false
                return Fm_2_bh_IO_nu(E)
            elseif nubar == true
                return Fm_2_bh_IO_nubar(E)
            else
                return 0.0
            end
        else
            return 0.0
        end
    else
        return 0.0
    end
end
function Fmass_test3(E, sm, ordering, nubar)
    if sm == "small"
        if ordering == "NO"
            if nubar == false
                return Fm_3_112sm_NO_nu(E)
            elseif nubar == true
                return Fm_3_112sm_NO_nubar(E)
            else
                return 0.0
            end
        elseif ordering == "IO"
            if nubar == false
                return Fm_3_112sm_IO_nu(E)
            elseif nubar == true
                return Fm_3_112sm_IO_nubar(E)
            else
                return 0.0
            end
        else
            return 0.0
        end
    elseif sm == "large"
        if ordering == "NO"
            if nubar == false
                return Fm_3_270sm_NO_nu(E)
            elseif nubar == true
                return Fm_3_270sm_NO_nubar(E)
            else
                return 0.0
            end
        elseif ordering == "IO"
            if nubar == false
                return Fm_3_270sm_IO_nu(E)
            elseif nubar == true
                return Fm_3_270sm_IO_nubar(E)
            else
                return 0.0
            end
        else
            return 0.0
        end
    elseif sm == "bh"
        if ordering == "NO"
            if nubar == false
                return Fm_3_bh_NO_nu(E)
            elseif nubar == true
                return Fm_3_bh_NO_nubar(E)
            else
                return 0.0
            end
        elseif ordering == "IO"
            if nubar == false
                return Fm_3_bh_IO_nu(E)
            elseif nubar == true
                return Fm_3_bh_IO_nubar(E)
            else
                return 0.0
            end
        else
            return 0.0
        end
    else
        return 0.0
    end
end

function Fmass(E, i, sm, ordering, nubar)
    if i == 1
        return Fmass_test1(E, sm, ordering, nubar)
    elseif i == 2
        return Fmass_test2(E, sm, ordering, nubar)
    elseif i == 3
        return Fmass_test3(E, sm, ordering, nubar)
    else
        return 0.0
    end
end;

In [8]:
# We can take normchoice to be SNRnorm, SNRnorm_low, or SNRnorm_high
function DSNB_integrand(E, z0, z, i, nubar, ordering, bh_frac, normchoice)
    c0 = 3e8 # m s^(-1)
    if bh_frac == "21"
        return 0.00208 * (c0*normchoice*SFR(z)/Hubble(z)) * (ηAD(15, 8)*Fmass(E*(1+z)/(1+z0), i, "small", ordering, nubar) 
        + (ηAD(22, 15)+ηAD(27, 25))*Fmass(E*(1+z)/(1+z0), i, "large", ordering, nubar) + (ηAD(25, 22)+ηAD(125, 27))*Fmass(E*(1+z)/(1+z0), i, "bh", ordering, nubar))
    elseif bh_frac == "41"
        return 0.00208 * (c0*normchoice*SFR(z)/Hubble(z)) * (ηAD(15, 8)*Fmass(E*(1+z)/(1+z0), i, "small", ordering, nubar) 
        + ηAD(125, 15)*Fmass(E*(1+z)/(1+z0), i, "bh", ordering, nubar))
    elseif bh_frac == "09"
        return 0.00208 * (c0*normchoice*SFR(z)/Hubble(z)) * (ηAD(15, 8)*Fmass(E*(1+z)/(1+z0), i, "small", ordering, nubar)
        + ηAD(40, 15)*Fmass(E*(1+z)/(1+z0), i, "large", ordering, nubar) + ηAD(125, 40)*Fmass(E*(1+z)/(1+z0), i, "bh", ordering, nubar))
    else
        return 0
    end
end

# Mass method
DSNB(E, z0, i, nubar, ordering, bh_frac, normchoice) = 1/(1+z0) * quadgk(z -> DSNB_integrand(E, z0, z, i, nubar, ordering, bh_frac, normchoice), z0, 5, rtol=1e-2)[1]

# νe method
function DSNB(E, z0, nubar, ordering, bh_frac, normchoice)
    ν1 = DSNB(E, z0, 1, nubar, ordering, bh_frac, normchoice)
    ν2 = DSNB(E, z0, 2, nubar, ordering, bh_frac, normchoice)
    ν3 = DSNB(E, z0, 3, nubar, ordering, bh_frac, normchoice)
    return Usqred(ordering)[1, 1]*ν1 + Usqred(ordering)[1, 2]*ν2 + Usqred(ordering)[1, 3]*ν3
end

DSNB (generic function with 2 methods)

In [209]:
# Seperating the progenitors: 1pc ≡ one progenitor contribution

function DSNB_integrand_1pc(E, z0, z, i, nubar, ordering, sm, normchoice)
    c0 = 3e8 # m s^(-1)
    # Adding in this integrated mass function so we can multiply by bh fractions at the end
    return 0.00208 * ηAD(125, 8) * (c0*normchoice*SFR(z)/Hubble(z)) * Fmass(E*(1+z)/(1+z0), i, sm, ordering, nubar)
end

DSNB_1pc(E, z0, i, nubar, ordering, sm, normchoice) = 1/(1+z0) * quadgk(z -> DSNB_integrand_1pc(E, z0, z, i, nubar, ordering, sm, normchoice), z0, 5, rtol=1e-2)[1]

function DSNB_1pc(E, z0, nubar, ordering, sm, normchoice)
    ν1 = DSNB_1pc(E, z0, 1, nubar, ordering, sm, normchoice)
    ν2 = DSNB_1pc(E, z0, 2, nubar, ordering, sm, normchoice)
    ν3 = DSNB_1pc(E, z0, 3, nubar, ordering, sm, normchoice)
    return Usqred(ordering)[1, 1]*ν1 + Usqred(ordering)[1, 2]*ν2 + Usqred(ordering)[1, 3]*ν3
end;

In [136]:
# Define this effective length function
LeffIntegrand(z) = (H0*sqrt(energy_matter*(1+z)^3 + energy_dark))^(-1)*(1+z)^(-2)

leff_quadint(z0) = quadgk(z -> LeffIntegrand(z), 0, z0)[1]

zs_leff = range(0, 5, 1000)
leff_ne = Interpolations.interpolate((vec(zs_leff),), leff_quadint.(zs_leff), Gridded(Linear()))
leff = extrapolate(leff_ne, 0.0)

function decay(E, α, z0, z)
    scalefactor = 4.68e28
    return exp(-scalefactor*α*(leff(z) - leff(z0))*(1+z0)/E)
end;

In [210]:
# Invisible decay

# mass method
function DSNB_idecay(E, z0, α, i, nubar, ordering, bh_frac, normchoice)
    return quadgk(z -> 1/(1+z0) * DSNB_integrand(E, z0, z, i, nubar, ordering, bh_frac, normchoice)*decay(E, α, z0, z), z0, 5, rtol=1e-3)[1]
end

# νe method
function DSNB_idecay(E, z0, α1, α2, α3, nubar, ordering, bh_frac, normchoice)
    ν1 = DSNB_idecay(E, z0, α1, 1, nubar, ordering, bh_frac, normchoice)
    ν2 = DSNB_idecay(E, z0, α2, 2, nubar, ordering, bh_frac, normchoice)
    ν3 = DSNB_idecay(E, z0, α3, 3, nubar, ordering, bh_frac, normchoice)
    return Usqred(ordering)[1, 1]*ν1 + Usqred(ordering)[1, 2]*ν2 + Usqred(ordering)[1, 3]*ν3
end

# Single progenitor versions
function DSNB_idecay_1pc(E, z0, α, i, nubar, ordering, sm, normchoice)
    return quadgk(z -> 1/(1+z0) * DSNB_integrand_1pc(E, z0, z, i, nubar, ordering, sm, normchoice)*decay(E, α, z0, z), z0, 5, rtol=1e-3)[1]
end

function DSNB_idecay_1pc(E, z0, α1, α2, α3, nubar, ordering, sm, normchoice)
    ν1 = DSNB_idecay_1pc(E, z0, α1, 1, nubar, ordering, sm, normchoice)
    ν2 = DSNB_idecay_1pc(E, z0, α2, 2, nubar, ordering, sm, normchoice)
    ν3 = DSNB_idecay_1pc(E, z0, α3, 3, nubar, ordering, sm, normchoice)
    return Usqred(ordering)[1, 1]*ν1 + Usqred(ordering)[1, 2]*ν2 + Usqred(ordering)[1, 3]*ν3
end;

In [43]:
# Energy spectrum function for SH case
function ψSH(Eh, El, hc)
    if hc
        return 2*El/Eh^2
    else
        return (2/Eh)*(1-(El/Eh))
    end
end;

In [212]:
# 2ν SH treatment assuming the smallest mass state has a mass of ≈0 (so all channels are SH except for IO ν2 → ν1, which is QD)

# We take m_j > m_i
function qcontrib_2ν(E, z0, z, j, jbar, i, ibar, αj, ordering, bh_frac, normchoice)

    if ordering == "NO" && j <= i
        return println("error: not kinematically allowed")
    elseif ordering == "IO" && j == 1 && i == 2
        return println("error: not kinematically allowed")
    elseif ordering == "IO" && j == 3 && i == 1
        return println("error: not kinematically allowed")
    elseif ordering == "IO" && j == 3 && i == 2
        return println("error: not kinematically allowed")
    elseif ordering == "IO" && j == 2 && i == 1
        if jbar != ibar
            return 0.0
        else
            Ers = E*(1+z)/(1+z0)
            qnorm = 3.086e19 * 1.516e15 * 1e6 / (3e8 * 1e12)
            return qnorm * (c0/Hubble(z))*DSNB_idecay(Ers, z, αj, j, jbar, ordering, bh_frac, normchoice) * (αj/Ers)
        end
    else
        if jbar == ibar
            hc = true
        else
            hc = false
        end

        Ers = E*(1+z)/(1+z0)

        qnorm = 3.086e19 * 1.516e15 * 1e6 / (3e8 * 1e12)
        integrand(Eprime) = qnorm * (c0/Hubble(z))*DSNB_idecay(Eprime, z, αj, j, jbar, ordering, bh_frac, normchoice) * (αj * 0.5/Eprime) * ψSH(Eprime, Ers, hc)
        Emax = Ers + 50

        return quadgk(Eprime -> integrand(Eprime), Ers, Emax, rtol=1e-2)[1]
    end
end

function DSNB_vdecay_2ν(E, αj, i, ibar, ordering, bh_frac, normchoice)
    if αj == 0
        return DSNB_idecay(E, 0, 0, i, ibar, ordering, bh_frac, normchoice)
    elseif ordering == "NO" && i == 3
        return DSNB_idecay(E, 0, αj, i, ibar, "NO", bh_frac, normchoice)
    elseif ordering == "IO" && i == 2
        return DSNB_idecay(E, 0, αj, i, ibar, "IO", bh_frac, normchoice)
    else
        # Here, for NO we take ν2 to be stable and for IO we take ν1 to be stable, and we take the lightest mass states to be stable as well
        if ordering == "NO"
            qint = quadgk(z -> (qcontrib_2ν(E, 0, z, 3, false, i, ibar, αj, "NO", bh_frac, normchoice)+qcontrib_2ν(E, 0, z, 3, true, i, ibar, αj, "NO", bh_frac, normchoice)), 0, 5, rtol=1e-2)[1]
        elseif ordering == "IO"
            qint = quadgk(z -> (qcontrib_2ν(E, 0, z, 2, false, i, ibar, αj, "IO", bh_frac, normchoice)+qcontrib_2ν(E, 0, z, 2, true, i, ibar, αj, "IO", bh_frac, normchoice)), 0, 5, rtol=1e-2)[1]
        else
            return println("error: ordering takes either 'NO' or 'IO'")
        end
        return DSNB_idecay(E, 0, 0, i, ibar, ordering, bh_frac, normchoice) + qint
    end
end;

# Other few cases where the heaviest mass state and lightest mass state are stable, and the middle mass state decays into the lightest mass state
function DSNB_vdecay_2ν_alt(E, αj, i, ibar, ordering, bh_frac, normchoice)
    if αj == 0
        return DSNB_idecay(E, 0, 0, i, ibar, ordering, bh_frac, normchoice)
    elseif ordering == "NO" && i == 2
        return DSNB_idecay(E, 0, αj, i, ibar, "NO", bh_frac, normchoice)
    elseif ordering == "IO" && i == 1
        return DSNB_idecay(E, 0, αj, i, ibar, "IO", bh_frac, normchoice)
    elseif ordering == "NO" && i == 3
        return DSNB_idecay(E, 0, 0, i, ibar, "NO", bh_frac, normchoice)
    elseif ordering == "IO" && i == 2
        return DSNB_idecay(E, 0, 0, i, ibar, "IO", bh_frac, normchoice)
    else
        # Here, for NO we take ν1 to be stable and for IO we take ν2 to be stable, and we take the lightest mass states to be stable as well
        if ordering == "NO"
            qint = quadgk(z -> (qcontrib_2ν(E, 0, z, 2, false, i, ibar, αj, "NO", bh_frac, normchoice)+qcontrib_2ν(E, 0, z, 2, true, i, ibar, αj, "NO", bh_frac, normchoice)), 0, 5, rtol=1e-2)[1]
        elseif ordering == "IO"
            qint = quadgk(z -> (qcontrib_2ν(E, 0, z, 1, false, i, ibar, αj, "IO", bh_frac, normchoice)+qcontrib_2ν(E, 0, z, 1, true, i, ibar, αj, "IO", bh_frac, normchoice)), 0, 5, rtol=1e-2)[1]
        else
            return println("error: ordering takes either 'NO' or 'IO'")
        end
        return DSNB_idecay(E, 0, 0, i, ibar, ordering, bh_frac, normchoice) + qint
    end
end

# daughter specifies the state the heaviest mass state decays into in our 2ν framework
function DSNB_vdecay_2ν_νe(E, α, daughter, ebar, ordering, bh_frac, normchoice)
    if ordering == "NO"
        if daughter == 1
            α1, α2, α3 = α, 0.0, α
        elseif daughter == 2
            α1, α2, α3 = 0.0, α, α
        else
            return println("error: for NO, 'daughter' must be either ν1 or ν2")
        end
    elseif ordering == "IO"
        if daughter == 3
            α3, α1, α2 = α, 0.0, α
        elseif daughter == 1
            α3, α1, α2 = 0.0, α, α
        else
            return println("error: for IO, 'daughter' must be either ν1 or ν3")
        end
    else
        return println("error: ordering must take either 'NO' or 'IO'")
    end
    ν3 = DSNB_vdecay_2ν(E, α3, 3, ebar, ordering, bh_frac, normchoice)
    ν2 = DSNB_vdecay_2ν(E, α2, 2, ebar, ordering, bh_frac, normchoice)
    ν1 = DSNB_vdecay_2ν(E, α1, 1, ebar, ordering, bh_frac, normchoice)
    return Usqred("NO")[1, 1]*ν1 + Usqred("NO")[1, 2]*ν2 + Usqred("NO")[1, 3]*ν3
end

# If we don't specify a daughter, the decay channel is assumed to be from the second lightest to the lightest state
function DSNB_vdecay_2ν_νe(E, α, ebar, ordering, bh_frac, normchoice)
    if ordering == "NO"
        α1, α2, α3 = α, α, 0.0
    elseif ordering == "IO"
        α1, α2, α3 = α, 0.0, α
    else
        return println("error: ordering must take either 'NO' or 'IO'")
    end
    ν3 = DSNB_vdecay_2ν_alt(E, α3, 3, ebar, ordering, bh_frac, normchoice)
    ν2 = DSNB_vdecay_2ν_alt(E, α2, 2, ebar, ordering, bh_frac, normchoice)
    ν1 = DSNB_vdecay_2ν_alt(E, α1, 1, ebar, ordering, bh_frac, normchoice)
    return Usqred("NO")[1, 1]*ν1 + Usqred("NO")[1, 2]*ν2 + Usqred("NO")[1, 3]*ν3
end


# Single progenitor versions
function qcontrib_2ν_1pc(E, z0, z, j, jbar, i, ibar, αj, ordering, sm, normchoice)

    if ordering == "NO" && j <= i
        return println("error: not kinematically allowed")
    elseif ordering == "IO" && j == 1 && i == 2
        return println("error: not kinematically allowed")
    elseif ordering == "IO" && j == 3 && i == 1
        return println("error: not kinematically allowed")
    elseif ordering == "IO" && j == 3 && i == 2
        return println("error: not kinematically allowed")
    elseif ordering == "IO" && j == 2 && i == 1
        if jbar != ibar
            return 0.0
        else
            Ers = E*(1+z)/(1+z0)
            qnorm = 3.086e19 * 1.516e15 * 1e6 / (3e8 * 1e12)
            return qnorm * (c0/Hubble(z))*DSNB_idecay_1pc(Ers, z, αj, j, jbar, ordering, sm, normchoice) * (αj/Ers)
        end
    else
        if jbar == ibar
            hc = true
        else
            hc = false
        end

        Ers = E*(1+z)/(1+z0)

        qnorm = 3.086e19 * 1.516e15 * 1e6 / (3e8 * 1e12)
        integrand(Eprime) = qnorm * (c0/Hubble(z))*DSNB_idecay_1pc(Eprime, z, αj, j, jbar, ordering, sm, normchoice) * (αj * 0.5/Eprime) * ψSH(Eprime, Ers, hc)
        Emax = Ers + 50

        return quadgk(Eprime -> integrand(Eprime), Ers, Emax, rtol=1e-2)[1]
    end
end

function DSNB_vdecay_2ν_1pc(E, αj, i, ibar, ordering, sm, normchoice)
    if αj == 0
        return DSNB_idecay_1pc(E, 0, 0, i, ibar, ordering, sm, normchoice)
    elseif ordering == "NO" && i == 3
        return DSNB_idecay_1pc(E, 0, αj, i, ibar, "NO", sm, normchoice)
    elseif ordering == "IO" && i == 2
        return DSNB_idecay_1pc(E, 0, αj, i, ibar, "IO", sm, normchoice)
    else
        # Here, for NO we take ν2 to be stable and for IO we take ν1 to be stable, and we take the lightest mass states to be stable as well
        if ordering == "NO"
            qint = quadgk(z -> (qcontrib_2ν_1pc(E, 0, z, 3, false, i, ibar, αj, "NO", sm, normchoice)+qcontrib_2ν_1pc(E, 0, z, 3, true, i, ibar, αj, "NO", sm, normchoice)), 0, 5, rtol=1e-2)[1]
        elseif ordering == "IO"
            qint = quadgk(z -> (qcontrib_2ν_1pc(E, 0, z, 2, false, i, ibar, αj, "IO", sm, normchoice)+qcontrib_2ν_1pc(E, 0, z, 2, true, i, ibar, αj, "IO", sm, normchoice)), 0, 5, rtol=1e-2)[1]
        else
            return println("error: ordering takes either 'NO' or 'IO'")
        end
        return DSNB_idecay_1pc(E, 0, 0, i, ibar, ordering, sm, normchoice) + qint
    end
end;

# Other few cases where the heaviest mass state and lightest mass state are stable, and the middle mass state decays into the lightest mass state
function DSNB_vdecay_2ν_alt_1pc(E, αj, i, ibar, ordering, sm, normchoice)
    if αj == 0
        return DSNB_idecay_1pc(E, 0, 0, i, ibar, ordering, sm, normchoice)
    elseif ordering == "NO" && i == 2
        return DSNB_idecay_1pc(E, 0, αj, i, ibar, "NO", sm, normchoice)
    elseif ordering == "IO" && i == 1
        return DSNB_idecay_1pc(E, 0, αj, i, ibar, "IO", sm, normchoice)
    elseif ordering == "NO" && i == 3
        return DSNB_idecay_1pc(E, 0, 0, i, ibar, "NO", sm, normchoice)
    elseif ordering == "IO" && i == 2
        return DSNB_idecay_1pc(E, 0, 0, i, ibar, "IO", sm, normchoice)
    else
        # Here, for NO we take ν1 to be stable and for IO we take ν2 to be stable, and we take the lightest mass states to be stable as well
        if ordering == "NO"
            qint = quadgk(z -> (qcontrib_2ν_1pc(E, 0, z, 2, false, i, ibar, αj, "NO", sm, normchoice)+qcontrib_2ν_1pc(E, 0, z, 2, true, i, ibar, αj, "NO", sm, normchoice)), 0, 5, rtol=1e-2)[1]
        elseif ordering == "IO"
            qint = quadgk(z -> (qcontrib_2ν_1pc(E, 0, z, 1, false, i, ibar, αj, "IO", sm, normchoice)+qcontrib_2ν_1pc(E, 0, z, 1, true, i, ibar, αj, "IO", sm, normchoice)), 0, 5, rtol=1e-2)[1]
        else
            return println("error: ordering takes either 'NO' or 'IO'")
        end
        return DSNB_idecay_1pc(E, 0, 0, i, ibar, ordering, sm, normchoice) + qint
    end
end

# daughter specifies the state the heaviest mass state decays into in our 2ν framework
function DSNB_vdecay_2ν_νe_1pc(E, α, daughter, ebar, ordering, sm, normchoice)
    if ordering == "NO"
        if daughter == 1
            α1, α2, α3 = α, 0.0, α
        elseif daughter == 2
            α1, α2, α3 = 0.0, α, α
        else
            return println("error: for NO, 'daughter' must be either ν1 or ν2")
        end
    elseif ordering == "IO"
        if daughter == 3
            α3, α1, α2 = α, 0.0, α
        elseif daughter == 1
            α3, α1, α2 = 0.0, α, α
        else
            return println("error: for IO, 'daughter' must be either ν1 or ν3")
        end
    else
        return println("error: ordering must take either 'NO' or 'IO'")
    end
    ν3 = DSNB_vdecay_2ν_1pc(E, α3, 3, ebar, ordering, sm, normchoice)
    ν2 = DSNB_vdecay_2ν_1pc(E, α2, 2, ebar, ordering, sm, normchoice)
    ν1 = DSNB_vdecay_2ν_1pc(E, α1, 1, ebar, ordering, sm, normchoice)
    return Usqred("NO")[1, 1]*ν1 + Usqred("NO")[1, 2]*ν2 + Usqred("NO")[1, 3]*ν3
end

# If we don't specify a daughter, the decay channel is assumed to be from the second lightest to the lightest state
function DSNB_vdecay_2ν_νe_1pc(E, α, ebar, ordering, sm, normchoice)
    if ordering == "NO"
        α1, α2, α3 = α, α, 0.0
    elseif ordering == "IO"
        α1, α2, α3 = α, 0.0, α
    else
        return println("error: ordering must take either 'NO' or 'IO'")
    end
    ν3 = DSNB_vdecay_2ν_alt_1pc(E, α3, 3, ebar, ordering, sm, normchoice)
    ν2 = DSNB_vdecay_2ν_alt_1pc(E, α2, 2, ebar, ordering, sm, normchoice)
    ν1 = DSNB_vdecay_2ν_alt_1pc(E, α1, 1, ebar, ordering, sm, normchoice)
    return Usqred("NO")[1, 1]*ν1 + Usqred("NO")[1, 2]*ν2 + Usqred("NO")[1, 3]*ν3
end;

In [239]:
# Function to save an N-dimensional array to a text file
function save_array_nd(array, filename)
    array2d = reshape(array, size(array, 1), prod(size(array)[2:end]))
    writedlm(filename, array2d)
end

# Function to load an N-dimensional array from a text file
# For dims, input size(array), where "array" was what you originally read into the text file
function load_array_nd(filename, dims)
    read_array2d = readdlm(filename, Int)
    reshaped_array = reshape(read_array2d, dims)
    return reshaped_array
end;

In [213]:
es_dsnb = range(0.5, 40, 100);

In [238]:
@time DSNB_vdecay_2ν_νe(1, 10^(-25), 3, true, "IO", "41", SNRnorm)

  0.091437 seconds (795.70 k allocations: 12.476 MiB)


0.43998003477509406

In [224]:
@time DSNB_vdecay_2ν_νe_1pc(1, 10^(-25), 3, true, "IO", "small", SNRnorm)

  0.060268 seconds (573.24 k allocations: 9.072 MiB)


0.6030759923836848

In [251]:
@time ηAD(15, 8)/ηAD(125, 8)*DSNB_vdecay_2ν_νe_1pc(1, 10^(-25), 3, true, "IO", "small", SNRnorm) + ηAD(125, 15)/ηAD(125, 8)*DSNB_vdecay_2ν_νe_1pc(1, 10^(-25), 3, true, "IO", "bh", SNRnorm)

  0.503784 seconds (1.24 M allocations: 19.593 MiB)


0.4399803007822404